<a href="https://colab.research.google.com/github/ljs7463/AnalysisProject/blob/main/%EB%8D%B0%EC%9D%B4%EC%BD%98/analytics/stacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import lib & load dataset

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os

# 시각화 폰트 설정
if os.name =='posix':
    plt.rc("font", family = "AppleGothic")

else:
    plt.rc("font", family = "Malgun Gothic")

# 경고문자 무시
warnings.filterwarnings(action='ignore')


# 데이터 로드
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_sub = pd.read_csv('sample_submission.csv')
df_info = pd.read_csv('data_info.csv')

# preprocessing & split dataset

In [3]:
## Label Encoding

## train데이터
df_train['preferred_difficulty_level'] = pd.factorize(df_train['preferred_difficulty_level'])[0]
df_train['subscription_type'] = pd.factorize(df_train['subscription_type'])[0]

## test데이터
df_test['preferred_difficulty_level'] = pd.factorize(df_test['preferred_difficulty_level'])[0]
df_test['subscription_type'] = pd.factorize(df_test['subscription_type'])[0]

# scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
for i in [['subscription_duration','recent_login_time','average_time_per_learning_session','monthly_active_learning_days','total_completed_courses','recent_learning_achievement','abandoned_learning_sessions','community_engagement_level','customer_inquiry_history','payment_pattern']]:
  df_train[i] = scaler.fit_transform(df_train[i])
for i in [['subscription_duration','recent_login_time','average_time_per_learning_session','monthly_active_learning_days','total_completed_courses','recent_learning_achievement','abandoned_learning_sessions','community_engagement_level','customer_inquiry_history','payment_pattern']]:
  df_test[i] = scaler.transform(df_test[i])

# Delete user_id
df_train = df_train.drop(columns = 'user_id')

# split target
x = df_train[list(df_train.columns[:-1])]
y = df_train['target']

# Delete user_id
df_test = df_test.drop(columns = 'user_id')
# split target
new_x = df_test


# import lib for modeling

In [ ]:
import optuna
from lightgbm import LGBMClassifier,early_stopping
from catboost import CatBoostClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import make_scorer, f1_score

In [ ]:
import optuna
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

# 사용할 모델 선정
# 1. lightgbm
# 2. catboost
# 3. logistic
# 4. SVM
# 5. KNN
# 6. 나이브베이즈



# catboost 하이퍼파라미터
cat_params= {
    'iterations': trial.suggest_int('iterations', 800, 1000),
    'depth': trial.suggest_int('depth', 4, 10),
    'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
    'random_strength': trial.suggest_int('random_strength', 0, 100),
    'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
    'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter'])
}


# LightGbm 하이퍼파라미터
params = {
      'objective': 'binary',
      'num_leaves': trial.suggest_int('num_leaves', 100, 500, step=1, log=True),
      'max_depth': trial.suggest_int('max_depth', 1, 10, step=1, log=False),
      'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
      'n_estimators': trial.suggest_int('n_estimators', 900, 1000, step=1, log=True),
      'class_weight': trial.suggest_categorical('class_weight', ['balanced', None]),
      'min_child_samples': trial.suggest_int('min_child_samples', 10, 50, step=10, log=False),
      'subsample': trial.suggest_uniform('subsample', 0.7, 1.0),
      'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 1.0),
      'reg_alpha': trial.suggest_uniform('reg_alpha', 0.0, 1.0),
      'reg_lambda': trial.suggest_uniform('reg_lambda', 0.0, 10.0),
      'random_state': 0
}

# xgboost 하이퍼파라미터
params = {
      "objective":'binary:logistic',
      'max_depth': trial.suggest_int('max_depth', 3, 9),
      'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
      'n_estimators': trial.suggest_int('n_estimators',1000,1001),
      'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
      'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
      'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
      'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
      'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
      'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
      'eval_metric': 'logloss',
      'use_label_encoder': False
  }
